<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [359]:
import pandas as pd
import psycopg2
import plotly.express as px
import plotly.graph_objects as go

In [360]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 


In [361]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [362]:
# текст запроса
query_3_1 = f'''
SELECT count(*) AS count_vacancies
FROM vacancies
'''


In [363]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\1114379196.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [364]:
# текст запроса
query_3_2 = f'''
SELECT count(*) AS count_employers
FROM employers
'''

In [365]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3062646138.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [366]:
# текст запроса
query_3_3 = f'''
SELECT count(*) AS count_areas
FROM areas
'''

In [367]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\928149064.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [368]:
# текст запроса
query_3_4 = f'''
SELECT count(*) AS count_industries
FROM industries
'''

In [369]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2101691108.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count_industries
0,294


***

In [370]:
# выводы по предварительному анализу данных

<ul>В исследуемой базе данных (БД) находится:
<li>вакансий - 49 197,</li>
<li>представлено работодателей - 23 501,</li>
<li>регионов - 1 362,</li>
<li>сфер деятельности - 294.</li>
</ul>
<ol>На текущий момент можно сделать несколько выводов:
<li>Структура БД логична, таблицы находятся во второй нормальной форме (2НФ), что вполне достаточно для комфортной работы с ними как в случае модификации данных, так и в случае анализа.</li>
<li>Есть неувязка с именами полей в таблицах: <i>area_id</i> в таблице vacancies и <i>area</i> в employers ссылаются на одно и то же поле <i>id</i> в таблице areas, но имеют различные имена.
<li>Регионы предствлены достаточно широко. По данным Википедии, в России 1132 города, включая города Федерального значения. Большее количество городов в БД может говорить о наличии вакансий в населенных пунктах, не имеющих статус города, либо о наличии зарубежных вакансий.</li>
<li>Объем данных вполне допускает проведение работ по анализу представленых вакансий.</li>
</ol>


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [371]:
# текст запроса
query_4_1 = f'''
SELECT
    a.name, -- наименование города
    COUNT(v.area_id) AS count_vacancies -- количество вакансий в городе
FROM areas a
    LEFT JOIN vacancies v ON a.id = v.area_id -- LEFT JOIN использован для получения информации по регионам, в которых нет вакансий
GROUP BY a.id -- группировка данных по идентификатору города
ORDER BY count_vacancies DESC -- сортировка результатов в порядке убывания
'''

In [372]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2457173557.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count_vacancies
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Балакирево,0
1358,Мексика,0
1359,Пугачев,0
1360,Долгоруково,0


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [373]:
# текст запроса
query_4_2 = f'''
SELECT
    COUNT(*) AS salary
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [374]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\970655580.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [375]:
# текст запроса
query_4_3 = f'''
SELECT
    ROUND(AVG(salary_from)) AS salary_from -- среднее значение нижней границы зарплаты
    ,ROUND(AVG(salary_to)) AS salary_to -- среднее значение верхней границы зарплаты
FROM vacancies
'''

In [376]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2500607024.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,salary_from,salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [377]:
# текст запроса
query_4_4 = f'''
SELECT
    schedule -- тип рабочего графика
    ,employment -- тип трудоустройства
    ,COUNT(*) AS vacancies -- количество вакансий для пары тип рабочего графика + тип трудоустройства
FROM vacancies
GROUP BY schedule,employment -- группировка по типу графика и типу трудоустройства, позволяет применить агрегатные функции к строкам, удовлетворяющим паре значений schedule,employment
ORDER BY vacancies DESC -- сортировка результата по убыванию
'''

In [378]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2022935265.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [379]:
# текст запроса
query_4_5 = f'''
SELECT
    experience -- опыт работы
    ,COUNT(*) AS vacancies -- количество вакансий для данного типа опыта работы
FROM vacancies
GROUP BY experience -- группировка по типу
ORDER BY vacancies -- сортировка по количеству вакансий в порядке возрастания
'''

In [380]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2316260449.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [381]:
# выводы по детальному анализу вакансий

<ol>
<li>общее количество представленных в БД регионов - 1 362, но только в 769 из них имеются вакансии;</li>
<li>зарплата указана у 24 073 вакансий, таким образом, более чем у половины вакансий зарплата не указана, что может повлиять на интерес к вакансии со стороны потенциальных кандидатов;</li>
<li>наиболее популярное сочетание типа рабочего графика и типа трудоустройства - "Полный день", "Полная занятость"; наименее популярное - "Вахтовый метод","Сменный график" в сочетании с "Проектная работа";</li>
<li>наиболее востребован опыт работы "От 1 года до 3 лет", наименее - "Более 6 лет".</li>
</ol>

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [382]:
# текст запроса
query_5_1 = f'''
SELECT
    e.name
    ,COUNT(*) as vacancies
FROM employers e
    JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.id
ORDER BY vacancies DESC
LIMIT 5
'''

In [383]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3834399874.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [384]:
# текст запроса
query_5_2 = f'''
SELECT
    a.name
    ,COUNT(DISTINCT e.id) AS employer -- количество работодателей в регионе
   ,COUNT(DISTINCT v.id) AS vacancies -- количество вакансий в регионе
FROM areas a
    JOIN employers e ON e.area = a.id
    LEFT JOIN vacancies v ON (v.area_id = a.id AND v.employer_id = e.id) -- LEFT JOIN для включения в результат тех регионов, где нет вакансий
GROUP BY a.id
ORDER BY employer DESC
'''

In [385]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2250345976.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,employer,vacancies
0,Москва,5864,4567
1,Санкт-Петербург,2217,1811
2,Минск,1115,1858
3,Алматы,721,1069
4,Екатеринбург,609,960
...,...,...,...
649,Кызылорда,1,1
650,Кизилюрт,1,0
651,Буйнакск,1,0
652,Грозный,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [386]:
# текст запроса
query_5_3 = f'''
SELECT
    e.name
    ,COUNT(DISTINCT a.id) AS area
FROM employers e
    JOIN vacancies v ON v.employer_id = e.id
    JOIN areas a ON v.area_id = a.id
GROUP BY e.id
ORDER BY area DESC
'''

In [387]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\201667002.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [388]:
# текст запроса
query_5_4 = f'''
SELECT
   COUNT(e.id)
FROM employers e
WHERE e.id NOT IN (SELECT employer_id FROM employers_industries)
'''

In [389]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3493105328.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [390]:
# текст запроса
query_5_5 = f'''
SELECT
   e.name
FROM employers e
    JOIN employers_industries ei ON ei.employer_id = e.id
GROUP BY e.id
HAVING COUNT(*) = 4 -- для включения в результат только те компании, у которых указано ровно четыре сферы деятельности
ORDER BY e.name
OFFSET 2 LIMIT 1 -- для получения только третьей по алфавиту компании
'''

In [391]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\388711819.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [392]:
# текст запроса
query_5_6 = f'''
SELECT
   COUNT(*)
FROM employers e
    JOIN employers_industries ei ON ei.employer_id = e.id
    JOIN industries i ON i.id = ei.industry_id
WHERE i.name = 'Разработка программного обеспечения'
'''

In [393]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\821628714.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [394]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # ссылка на статью в Wiki
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
city_list=[] # список под города
tables = page.find_all('table') # поиск всех тегов table
for elements in tables:
    if elements.find('th', text = '№') is not None: # поиск нужной таблицы по заголовку первого столбца
        tr_table = elements.find_all('tr')[1:] # поиск строк с названиями городов (опускаем строку-заголовок)
        for tr in tr_table:
            td = tr.find_all('td')[1].next_element # поиск всех ячеек в строке с выбором второй ячейки
            city_list.append(td['title']) # добавление наименования города в список
        break
city_str =  ', '.join("'"+city+"'" for city in city_list) # получение строки для вставки в запрос

In [395]:
# текст запроса
query_5_7 = f'''
SELECT
    a.name
    ,COUNT(*) AS cnt
FROM areas a
    JOIN vacancies v ON v.area_id = a.id
    JOIN employers e ON e.id = v.employer_id
WHERE a.name IN ({city_str}) -- подстановка списка городов
    AND e.name = 'Яндекс'
GROUP BY a.id
UNION -- для включения в отчет итоговой информации
SELECT
    'Total' AS name
    ,COUNT(*) AS cnt
FROM areas a
    JOIN vacancies v ON v.area_id = a.id
    JOIN employers e ON e.id = v.employer_id
WHERE a.name IN ({city_str})
    AND e.name = 'Яндекс'
ORDER BY cnt
'''

In [396]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\1965596796.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [397]:
# выводы по анализу работодателей

<ol>
<li>Пятерку работодателей по представленным вакансиям с большим отрывом возглавляет Яндекс;</li>
<li>Первые три компании известны активным развитием информационных технологий;</li>
<li>количество вакансий у первой пятерки дает более 7% от общего числа вакансий, Яндекс предлагает чуть меньше 4% вакансий от общего числа;</li>
<li>в 300 регионах есть работодатели, но нет вакансий, большая часть таких регионов не относится к населенным пунктам, а охватывает обширную территорию (Россия, Казахстан, Московская область и т.д.);</li>
<li>Яндекс и Ростелеком публикуют свои заявки в более чем 150 регионах, что достаточно логично, учитывая масштаб этих компаний, в первой пятерке по количеству охваченных регионов удивительно только наличие работодателя Поляков Денис Иванович, который размещает заявки в 88 регионах;</li>
<li>8 419 работодателей не указали сферу деятельности, что составляет более одной трети от их общего числа; это может отрицательно сказаться на интересе потенциальных кандидатов к данным работодателям и их вакансиям;</li>
<li>3 553 работодателя указали в качестве сферы деятельности "Разработку программного обеспечения";</li>
<li>Яндекс 485 (25%) заявок из 1933 размещает в 16 городах-миллионниках, но, если посмотреть на результаты чуть шире, то можно предположить, что количество вакансий примерно пропорционально числу жителей региона, т.е. Яндекс не сосредотачивается только на крупных городах.</li>
</ol>

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [398]:
# текст запроса
query_6_1 = f'''
SELECT
    COUNT(*) AS cnt
FROM vacancies v
WHERE LOWER(v.name) LIKE ANY(ARRAY['%data%','%данн%'])
'''

In [399]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\366702258.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [400]:
# текст запроса
query_6_2 = f'''
SELECT
    COUNT(*) AS cnt
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND
    (
        LOWER(v.name) LIKE '%junior%'
        OR v.experience = 'Нет опыта'
        OR v.employment = 'Стажировка'
    )
'''

In [401]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2267038862.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [402]:
# текст запроса
query_6_3 = f'''
SELECT
    COUNT(*) AS cnt
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND LOWER(v.key_skills) LIKE ANY(ARRAY['%sql%','%postgres%'])
'''

In [403]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3794299702.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [404]:
# текст запроса
query_6_4 = f'''
SELECT
    COUNT(*) AS cnt
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND v.key_skills LIKE '%Python%'
'''

In [405]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\4264665179.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [406]:
# текст запроса
query_6_5 = f'''
SELECT
    ROUND(AVG(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(REPLACE(v.key_skills,E'\t',''))+1),2) AS length
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
        AND v.key_skills IS NOT NULL
'''

In [407]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3654210517.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,length
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [408]:
# текст запроса
query_6_6 = f'''
SELECT
    v.experience
    ,ROUND(AVG((COALESCE(v.salary_from,v.salary_to) + COALESCE(v.salary_to,v.salary_from))/2),0) AS salary
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND NOT (v.salary_from IS NULL AND v.salary_to IS NULL)
GROUP BY v.experience
'''

In [409]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\519194770.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [410]:
# выводы по предметному анализу


<ol>
<li>1 771 вакансия имеет отношения к работе с данными;</li>
<li>на 51 вакансию могут претендовать начинающие дата-сайентисты (DS);</li>
<li>в 201 вакансии для DS в качестве ключевого навыка указаны SQL или postgres;</li>
<li>в 351 вакансии для DS в качестве ключевого навыка указан Python;</li>
<li>в среднем для DS в вакансиях указывается 6.41 ключевых навыка;</li>
<li>средняя зарплата для DS в зависимости от опыта работы составляет от 74 643 руб. при отсутствии опыта до 243 115 руб. при опыте работы от 3 до 6 лет; для опыта работы более 6 лет в вакансиях заработная плата не указывается.</li>
</ol>
Можно заметить, что количество вакансий для начинающих DS составляет весьма скромную долю от числа вакансий, потенциально относящихся к DS.

# Общий вывод по проекту

In [411]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

<ol>
<li>В базе данных представлено большое число работодателей из большинства регионов не только России, но и ближнего и дальнего зарубежья;</li>
<li>работодатели далеко не всегда указывают свои сферы деятельности, а также предлагаемую зарплату, это достаточно странно, т.к. делает данные вакансии менее заметными на общем фоне;</li>
<li>часть работодателей указывает в качестве своего региона не город, а обширную территорию, например, Россию, единственный смысл в этом можно увидеть, если эти работодатели предлагают в качестве варианта "Удаленную работу"</li>
<li>по количеству вакансий и предлагаемой ЗП лидирует ожидаемо Москва, количество предлагаемых по регионам вакансий похоже на некоторую долю от активного населения, а не от приближенности к Москве;</li>
<li>доля вакансий, относящихся к работе с данными, в общем кочичестве занимает лишь небольшую часть;</li>
<li>начинающим DS предстоит выбирать всего из 51 вакансии;</li>
<li>среднее количество ключевых навыков для DS составляет 6.41, было бы интересно изучить данные подробней.
</ol>

В вакансиях всегда возникает вопрос о ключевых навыках. Иногда работодатели выдвигают очень странные требования к ключевым навыкам соискателей.<br />
Необходимо провести исследования, которые позволят лучше понять не только соответствие потенциального соискателя рассматриваемой вакансии, но и получить некоторое представление о работодателе.<br />
Ниже будет проведено исследование ключевых навыков, указанных в вакансиях для DS.

In [412]:
query_skill_1 = f'''
SELECT
    e.name AS employer -- работодатели
    ,v.name AS vacancies -- вакансии
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
        AND v.key_skills IS NULL -- только те вакансии, где отсутствуют ключевые навыки
'''

df = pd.read_sql_query(query_skill_1, connection)

display('Вакансии, в которых не заполнены ключенвые навыки')
display(df)

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\652520571.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



'Вакансии, в которых не заполнены ключенвые навыки'

,employer,vacancies
0,Сталепромышленная компания,Специалист Data Science (R)
1,"БИОКАД, биотехнологическая компания",Data Scientist
2,СБЕР,Researcher Data Science
3,Эр-1,Data Scientist
4,Эр-1,Data Scientist
5,"ALSECO, АО",ML-инженер/разработчик NLP
6,Homework,Автор работ по машинному обучению
7,Гринатом,Data Scientist
8,МСП Банк,Data Scientist
9,Ozon,Математик-разработчик (Data Scientist)


In [413]:
query_skill_2 = f'''
SELECT
    MAX(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(REPLACE(v.key_skills,E'\t',''))+1) -- максимальное число навыков в вакансии
    ,MIN(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(REPLACE(v.key_skills,E'\t',''))+1) -- минимальное число навыков в вакансии
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND v.key_skills IS NOT NULL -- только те вакансии, где указаны ключевые навыки
'''

df = pd.read_sql_query(query_skill_2, connection)

display('Максимальное и минимальное количество навыков, указанных в вакансиях')
display(df)

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2075601696.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



'Максимальное и минимальное количество навыков, указанных в вакансиях'

,max,min
0,20,1


In [414]:
query_skill_3 = f'''
SELECT
    e.name AS employer
    ,v.name AS vacancies
    ,REPLACE(v.key_skills,E'\t',', ') AS skills -- замена в строке с навыками табуляции на запятую
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
        AND v.key_skills IS NOT NULL
        
    AND v.key_skills LIKE '%Python%'
'''

df = pd.read_sql_query(query_skill_3, connection)

dic_skills = dict() # словарь, в который будут записываться навыки в качестве ключа и количество их упоминаний в качестве значения

for skill in df['skills']: # проход по всем строкам и столбцу 'skills'
    values = skill.split(', ') # разбиение строки по запятой, получение списка навыков
    for i in range(len(values)):
        dic_skills[values[i]] = dic_skills.get(values[i],0)+1 # увеличение значения на 1 для соответствующего ключа словаря

# получение из словаря DataFrame с двумя столбцами (навык, количество упоминаний) с сортировкой по количеству упоминаний в порядке уменьшения
df_skills = pd.DataFrame(dic_skills.items(), columns=['skill','value']).sort_values(by='value',ascending=False)

display('Первые десять по популярности ключевых навыков, встречающихся в вакансиях для DS')
display(df_skills.head(10))

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\239335859.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



'Первые десять по популярности ключевых навыков, встречающихся в вакансиях для DS'

,skill,value
0,Python,348
1,SQL,165
7,Machine Learning,109
25,Математическая статистика,60
9,Git,55
56,Pandas,51
40,Data Science,50
35,Data Analysis,50
4,ML,48
55,PyTorch,46


In [415]:
query_skill_4 = f'''
SELECT
    MAX(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(REPLACE(v.key_skills,E'\t',''))+1) -- максимальное число навыков в вакансии
    ,MIN(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(REPLACE(v.key_skills,E'\t',''))+1) -- минимальное число навыков в вакансии
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND e.name LIKE ANY(ARRAY['Яндекс','Ростелеком','Тинькофф','СБЕР']) -- включим в запрос только первую четверку лидеров по количеству вакансий
    AND v.key_skills IS NOT NULL
'''

df = pd.read_sql_query(query_skill_4, connection)

display('Максимальное и минимальное количество навыков, указанных в вакансиях для DS от четверки работодателей')
display(df)

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\598165036.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



'Максимальное и минимальное количество навыков, указанных в вакансиях для DS от четверки работодателей'

,max,min
0,17,1


In [416]:
query_skill_5 = f'''
SELECT
    e.name AS employer
    ,v.name AS vacancies
    ,REPLACE(v.key_skills,E'\t',', ') AS skills -- замена в строке с навыками табуляции на запятую
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND e.name LIKE ANY(ARRAY['Яндекс','Ростелеком','Тинькофф','СБЕР']) -- включим в запрос только первую четверку лидеров по количеству вакансий
    AND v.key_skills IS NOT NULL
'''

df = pd.read_sql_query(query_skill_5, connection)

dic_skills = dict() # словарь, в который будут записываться навыки в качестве ключа и количество их упоминаний в качестве значения

for skill in df['skills']: # проход по всем строкам и столбцу 'skills'
    values = skill.split(', ') # разбиение строки по запятой, получение списка навыков
    for i in range(len(values)):
        dic_skills[values[i]] = dic_skills.get(values[i],0)+1 # увеличение значения на 1 для соответствующего ключа словаря

# получение из словаря DataFrame с двумя столбцами (навык, количество упоминаний) с сортировкой по количеству упоминаний в порядке уменьшения
df_skills = pd.DataFrame(dic_skills.items(), columns=['skill','value']).sort_values(by='value',ascending=False)

display('Первые десять по популярности ключевых навыков, встречающихся в вакансиях для DS от четверки лидеров')
display(df_skills.head(10))

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\2000919780.py:17: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



'Первые десять по популярности ключевых навыков, встречающихся в вакансиях для DS от четверки лидеров'

,skill,value
0,Python,33
11,SQL,14
36,Spark,10
7,ML,10
15,Machine Learning,8
4,Java,8
17,Hadoop,8
31,Pandas,7
14,Big Data,6
23,Машинное обучение,5


# Выводы по исследованию ключевых навыков
<ul>
<li>вакансии, в которых не указаны ключевые навыки, ориентированы на опытных DS, которым можно лишний раз не напоминать о том, что они должны знать и уметь;</li>
<li>максимальный набор ключевых навыков составляет 20 единиц для DS и 14 для начинающих DS;</li>
<li>если обратиться к наиболее популярным навыкам, то лидеры для DS практически одинаковы, независимо от опыта работы: Python, SQL и Machine Learning;</li>
<li>для четырех наиболее активных работодателей в лидерах по ключевым навыкам те же Python, SQL и Machine Learning;</li>
<li>наличие ярко выраженных лидеров в списке ключевых данных показывает, что переход из начинающих DS к более высоким должностям зависит в большей степени от опыта работы, нежели от наличия специфических навыков;</li>
<li>очень часто в одной вакансии присутствуют навыки, близкие по функционалу, что связано, видимо, с попыткой работодателя охватить максимальный набор ключевых слов, по которым идет поиск;</li>
<li>хоть и не часто, но встречаются навыки, которые не имеют прямого отношения к DS, например, Adobe Photoshop или MS PowerPoint.</li>
</ul>
Анализ вакансий однозначно указывает на приоритет в навыках: Python, SQL и Machine Learning. Языки программирования - лидеры прошлых лет (C++, C#, Java) уступают активно развивающемуся Python, к плюсам которого несомненно стоит отнести большое количество библиотек, позволяющих легко решать самые разнообразные задачи. SQL и Machine Learning должны и занимают высокие позиции в требованиях для DS.



In [417]:
# вспомогательныйй запрос, возвращающий наименования поля experience - требования к опыту для DS
query_skill_6 = f'''
SELECT
    v.experience
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND NOT (v.salary_from IS NULL AND v.salary_to IS NULL) -- т.к. дальнейшие расчеты будут идти для тех вакансий, по которым указана ЗП, то остальные данные исклюсаются
GROUP BY v.experience
'''
df_experience = pd.read_sql_query(query_skill_6, connection)

# осноаной запрос
query_skill_7 = f'''
SELECT
    v.experience
    ,ROUND((COALESCE(v.salary_from,v.salary_to) + COALESCE(v.salary_to,v.salary_from))/2,0) AS salary
FROM vacancies v
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND NOT (v.salary_from IS NULL AND v.salary_to IS NULL)
'''
# визуализация полученных данных
df = pd.read_sql_query(query_skill_7, connection)
fig = go.Figure()
for experience in df_experience['experience']:
    fig.add_trace(go.Box(x=df[df['experience'] == experience]['salary'],name=experience))
    fig.update_layout(title="Распределение заработной платы в зависимости от опыта",
                  yaxis_title="Опыт",
                  xaxis_title="ЗП")
fig.show()
fig.write_html("pic01.html")

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\935443850.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\935443850.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [418]:
# ранее был получен список городов-миллионников с сайта wikipedia. Запрос даст оценку ЗП по городам для DS
query_skill_8 = f'''
SELECT
    a.name AS city
    ,ROUND(AVG((COALESCE(v.salary_from,v.salary_to) + COALESCE(v.salary_to,v.salary_from))/2),0) AS salary
FROM vacancies v
    JOIN areas a ON v.area_id = a.id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND a.name in ({city_str})
    AND NOT (v.salary_from IS NULL AND v.salary_to IS NULL)
GROUP BY a.id
ORDER BY salary DESC
'''

df = pd.read_sql_query(query_skill_8, connection)

# визуализация полученных данных
fig = px.bar(
    data_frame=df, #датафрейм
    x="city", #ось x
    y="salary", #ось y
    text = 'salary', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Зависимость ЗП от города' #заголовок
)

fig.show()
fig.write_html("pic02.html")

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\4163003935.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [419]:
# запрос созвращает список городов и ЗП, которая предлагается в этих городах для начинающих DS
query_skill_9 = f'''
SELECT
    a.name AS city
    ,ROUND(AVG((COALESCE(v.salary_from,v.salary_to) + COALESCE(v.salary_to,v.salary_from))/2),0) AS salary
FROM vacancies v
    JOIN areas a ON v.area_id = a.id
WHERE
    (
        LOWER(v.name) LIKE ANY(ARRAY['%data scientist%','%data science%','%исследователь данных%','%machine learning%','%машинн%обучен%'])
        OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    )
    AND a.name in ({city_str})
    AND NOT (v.salary_from IS NULL AND v.salary_to IS NULL)
    AND
    (
        LOWER(v.name) LIKE '%junior%'
        OR v.experience = 'Нет опыта'
        OR v.employment = 'Стажировка'
    )
GROUP BY a.id
ORDER BY salary DESC
'''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query_skill_9, connection)

fig = px.bar(
    data_frame=df, #датафрейм
    x="city", #ось x
    y="salary", #ось y
    text = 'salary', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Зависимость ЗП от города для начинающих DS' #заголовок
)

fig.show()
fig.write_html("pic03.html")

C:\Users\mmvvs\AppData\Local\Temp\ipykernel_6952\3863251408.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



<ul>
<li>изучение данных по ЗП подтверждает то, что опыт положительно сказывается на предлагаемой ЗП;</li>
<li>диаграммы получаются без выбросов;</li>
<li>опыт работы от 3 до 6 лет дает хорошую ЗП по отношению к другим категориям опыта, но при этом можно заметить, что нижний ус диаграммы уходит ниже медианы зарплаты без опыта;</li>
<li>медиана ЗП с опытом работы от 3 до 6 лет расположена ближе к левому краю диаграммы;</li>
<li>другие варианты опыта работы более компакты по предлагаемой ЗП;</li>
<li>связан такой разброс, скорее всего, с большим различием предлагаемых должностей и требуемых навыков;</li>
<li>средняя зарплата DS составляет в Москве 207 630 руб., более 100 000 руб. предлагают еще работодатели из Санкт-Петербурга, Новосибирска и Екатеринбурга;</li>
<li>связано это, скорее всего, с тем, что эти города являются научными центрами, соответственно, существует конкуренция между работодателями;</li>
<li>начинающим DS лучше всего набираться опыта в Санкт-Петербурге или Новосибирске, Московские работодатели находятся лишь на 3 месте по уровню предлагаемых ЗП для начинающих DS.
<ul>


In [420]:
connection.close()